In [48]:
import cv2
import json
import os
import numpy as np

from shutil import copyfile

In [49]:
root_dir=r"C:\Users\108890\Desktop\mta\done"
save_dir=r"C:\Users\108890\Desktop\mta\tmp"

postfix=".json"
image_postfix=".jpg"

#水平边上点的个数
horizon_keypoint_num = 25

#垂直边上点的个数
vertical_keypoint_num = 15

In [50]:
def __get_files_recursive(fd, suffix=None):
    fs, ds = [], []
    for root, dirs, files in os.walk(fd):
        for name in files:
            if suffix is None:
                fs.append(os.path.join(root, name))
            else:
                if name.endswith(suffix):
                    fs.append(os.path.join(root, name))
    return fs

In [51]:
def write_pts(pts, dst):
    with open(dst, 'w') as f:
        f.write("version:1\n")
        # TODO: change 56 to the number of points you want
        f.write("n_points :{}\n".format(len(pts)))
        f.write("{ \n")
        for x, y in pts:
            f.write("{} {}\n".format(x, y))
        f.write("} \n")
        f.write(" ")

In [52]:
def run(root_dir, postfix, image_postfix, horizon_keypoint_num, vertical_keypoint_num, save_dir=None):

    horizon_keypoint_num = horizon_keypoint_num - 1
    vertical_keypoint_num = vertical_keypoint_num - 1
    
    json_list = __get_files_recursive(root_dir, postfix)#[i for i in os.listdir(root_dir) if '.json' in i]
    for json_path in json_list:
        try:
            with open(json_path, 'r') as f:
                line = []
                data = json.load(f)
                points = data['shapes'][0]['points']
                
                img = data['imagePath']
                mx = 0
                my = 0
                for pt in points:
                    x, y = pt
                    mx = mx + int(x)
                    my = my + int(y)
                    line.append([int(x), int(y)])
                mx = mx / 4
                my = my / 4
                for pt in points:
                    x, y = pt
                    if int(x) < mx and int(y) < my:
                        line[0] = [int(x), int(y)]
                    elif int(x) < mx and int(y) > my:
                        line[3] = [int(x), int(y)]
                    elif int(x) > mx and int(y) < my:
                        line[1] = [int(x), int(y)]
                    elif int(x) > mx and int(y) > my:
                        line[2] = [int(x), int(y)]
                line.append(line[0])
            pts = []
            if False: # kp_numbers_cfg['cnt'] == 4:
                for x, y in line:
                    pts.append(x, y)
            else:
                for i in range(len(line) - 1):
                    # TODO: 20 is horizontal numbers, 10 is vertical numbers  #kp_numbers_cfg['edge1'] horizon
                    delta = horizon_keypoint_num + 1 if i % 2 == 0 else vertical_keypoint_num + 1
                    if i == 0:
                        startx, starty = line[i - 2]
                    else:
                        startx, starty = line[i - 1]
                    startx, starty = line[i]
                    endx, endy = line[i+1]
                    dx = int((endx - startx) / delta)
                    dy = int((endy - starty) / delta)
                    pts.append(points[i])
                    for i in range(1, delta): #0
                        tx = startx + i * dx
                        ty = starty + i * dy
                        pts.append([int(tx), int(ty)])
                        #print([int(tx), int(ty)])
            if save_dir is None:
                write_pts(pts, json_path.replace(postfix, '.pts'))
            else:
                json_name = os.path.basename(json_path)
                write_pts(pts, os.path.join(save_dir, json_name.replace(postfix, '.pts')))
                copyfile(json_path.replace(postfix, image_postfix), os.path.join(save_dir, json_name.replace(postfix, image_postfix)))
        except:
            print(json_name)

In [57]:
if __name__ == '__main__':
    print("start ...")
    run(root_dir, postfix, image_postfix, horizon_keypoint_num, vertical_keypoint_num, save_dir=None)
    print("finish ...")

start ...
finish ...
